In [ ]:
import os 

email = os.environ["EMAIL"]
!wget -O sibbr.zip "https://biocache-service.sibbr.gov.br/biocache-service/occurrences/index/download?q=*%3A*&email=${email}&sourceTypeId=&reasonTypeId=3&file=data&extra=dataResourceUid,dataResourceName.p"

In [10]:
!unzip -d sibbr/ sibbr.zip 

Archive:  sibbr.zip
  inflating: sibbr/data.csv          
  inflating: sibbr/data_part2.csv    
  inflating: sibbr/data_part3.csv    
  inflating: sibbr/data_part4.csv    
  inflating: sibbr/citation.csv      
  inflating: sibbr/README.html       
  inflating: sibbr/headings.csv      


In [11]:
%load_ext autoreload
%autoreload 2
import duckdb
import os
import pandas as pd
import pathlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
local_file = "./sibbr_parquet"

### Convert to parquet

In [13]:
os.makedirs(local_file, exist_ok=True)

for p in pathlib.Path("./sibbr").rglob("data*.csv"):
    print(p)
    df = pd.read_csv(p)

    for column in df.columns:
        if df[column].dtype == "object":
            df[column] = df[column].astype(str)

    filename = os.path.join(local_file, "%s.parquet" % p.stem)
    df.to_parquet(filename)

sibbr/data_part3.csv


/tmp/ipykernel_5764/3701533431.py:5: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p)


sibbr/data_part2.csv
sibbr/data.csv


/tmp/ipykernel_5764/3701533431.py:5: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p)


sibbr/data_part4.csv


In [14]:
conn = duckdb.connect("sibbr.db")

conn.execute("DROP TABLE IF EXISTS sibbr")
conn.execute(f"""
        CREATE TABLE sibbr AS
        SELECT * FROM read_parquet('{local_file}', union_by_name=True)
    """)

result = conn.execute("SELECT COUNT(*) FROM sibbr").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Registros carregados na tabela: 10,000,000
